In [1]:
from os import listdir
from os.path import isfile, join
seed_path="./assets/cases/purchaseorder"
onlyfiles = [f for f in listdir(seed_path) if isfile(join(seed_path, f)) and f.endswith('.xml')]
onlyfiles

['Purchase_order_test_data.xml']

In [11]:
import os
os.path.basename(seed_path)

'purchaseorder'

In [2]:
import xml.etree.ElementTree as ET

def collect_types(f):
    xml_path=join(seed_path, f)
    tree = ET.parse(xml_path)
    root = tree.getroot()
    ent_types=set([c.tag for c in root])
    return ent_types

all_types=set()
for f in onlyfiles:
    all_types.update(collect_types(f))

len(all_types), all_types

(12,
 {'ItemIssuance',
  'OrderContactMech',
  'OrderHeader',
  'OrderItem',
  'OrderItemPriceInfo',
  'OrderItemShipGroup',
  'OrderItemShipGroupAssoc',
  'OrderRole',
  'OrderStatus',
  'Payment',
  'Shipment',
  'ShipmentItem'})

In [3]:
from meta_generator import metagen
pkg='purchaseorder'
metagen.gen_ents(pkg, all_types, pkg)

2021-06-10 03:37:33,874 - sagas.ofbiz.connector - INFO - connect to py4j-gateway localhost 22333


In [8]:
from sagas.ofbiz.entities import entity
edges=[]
for typ in all_types:
    entity_meta=entity(typ)
    model=entity_meta.model
    relations=[{"type": rel.getType(),
                "rel": rel.getRelEntityName()}
               for rel in model.getRelations() if rel.getRelEntityName() in all_types]
    edges.append({"name":typ, "relations": relations})

edges    

[{'name': 'ShipmentItem',
  'relations': [{'type': 'one', 'rel': 'Shipment'},
   {'type': 'many', 'rel': 'ItemIssuance'}]},
 {'name': 'OrderItem',
  'relations': [{'type': 'one', 'rel': 'OrderHeader'},
   {'type': 'many', 'rel': 'ItemIssuance'},
   {'type': 'many', 'rel': 'OrderItemPriceInfo'},
   {'type': 'many', 'rel': 'OrderItemShipGroupAssoc'},
   {'type': 'many', 'rel': 'OrderStatus'}]},
 {'name': 'OrderItemShipGroupAssoc',
  'relations': [{'type': 'one', 'rel': 'OrderHeader'},
   {'type': 'one', 'rel': 'OrderItem'},
   {'type': 'one', 'rel': 'OrderItemShipGroup'}]},
 {'name': 'ItemIssuance',
  'relations': [{'type': 'one-nofk', 'rel': 'Shipment'},
   {'type': 'one', 'rel': 'ShipmentItem'},
   {'type': 'one-nofk', 'rel': 'OrderHeader'},
   {'type': 'one', 'rel': 'OrderItem'}]},
 {'name': 'OrderContactMech',
  'relations': [{'type': 'one', 'rel': 'OrderHeader'}]},
 {'name': 'OrderStatus',
  'relations': [{'type': 'one', 'rel': 'OrderHeader'},
   {'type': 'one-nofk', 'rel': 'OrderIt